# K-Means Clustering Evaluation

This Python notebook is used for evaluation of a dictionary that is produced by:

- Find the cluster a word belongs to 
- Find the other words in the cluster for a specific word
- Compare between clusters

In [1]:
import pickle
import random

# Specify the files
FILE_DICT_500 = "C:/Users/MyPC/Desktop/Vegito/Word Dictionaries/dict_500C.pk"
FILE_CLUS_500 = "C:/Users/MyPC/Desktop/Vegito/K-Means Models/full_500C.pk"

# Load using pickle
array_dict_cluster_500 = pickle.load(open(FILE_DICT_500, "rb"))
word_centroid_map_500 =  pickle.load(open(FILE_CLUS_500,"rb"))

total_clusters_500 = max(word_centroid_map_500.values()) + 1

average_word_500 = round(len(word_centroid_map_500)/total_clusters_500)

In [38]:
# Find the cluster of words, based on a given word
search = input("ENTER WORD TO SEARCH: ")

# Get the key, or cluster number
# NOTE: Different clusters can have same results
cluster_num_500 = word_centroid_map_500[search]

# Return the array based on the cluster number
words_500 = array_dict_cluster_500[cluster_num_500]['word_list']

# Randomly subsample 200 words
words = random.sample(words_500, 200)

# Display results
print("SEARCHED WORD: %s \n" % (search))

print("TOTAL WORDS: %i \n" % (len(words_500)))

# Pretty Printing
for i, word in enumerate(words):
    
    if i % 5 != 0:
        print(word, " ", end="")
    else:
        print("\n")

ENTER WORD TO SEARCH: kimchee
SEARCHED WORD: kimchee 

TOTAL WORDS: 1565 



feijoada  bechamel  jarlsberg  fries  

makhani  tonkotsu  flautas  potpie  

toppings  sweetcorn  parsnip  cubed  

veg  marinades  perrins  chipotles  

oyster  appetizers  egss  sauceless  

lunchmeat  bisque  kofta  blanch  

microwaveable  umami  chops  marzu  

kapsalon  cheddars  scrambled  filet  

breakfasty  haggis  babyback  kimchee  

rosemary  tefal  cookery  loaf  

freschetta  marinara  peperoni  flatbread  

tofurkey  teryaki  basashi  seafood  

guajillo  serranos  omelets  yuca  

marinading  mignon  laksa  kimbap  

rotisserie  mussels  yam  sourdough  

beetroot  crockpot  kohlrabi  fraiche  

horseradish  oxtail  quesadillas  riblets  

jalapenos  livermush  bleu  entree  

nutribullet  sundried  berbere  saucepan  

frittata  baozi  sauteeing  piccata  

gefilte  tenderizing  dollop  calrose  

tarragon  bannock  sesame  mapo  

grater  schmaltz  havarti  zoodles  

spinach  sausages  bea